In [103]:
import json
import datetime

def read_file(path):
    fObj = open(path)
    objectData = json.load(fObj)
    return objectData

def toDateTime(str):
    return datetime.datetime.strptime(str, '%Y-%m-%dT%H:%M:%S+00:00')

In [104]:
class DataPoint:
    def __init__(self, time, value, label = None):
        self.time = time
        self.value = value
        self.label = label 

class FileParser:
    def __init__(self):
        self.metricsByLabel = {}
        pass

    def parse(self, jsonObj, labelPrefix):
        metrics = jsonObj['MetricDataResults']
        for dataSet in metrics:
            label = labelPrefix + dataSet['Label']
            timeStamps = dataSet['Timestamps']
            values = dataSet['Values']
            dataPointsByTime = {}
            for i in range(len(timeStamps)):
                t = toDateTime(timeStamps[i])
                v = values[i]
                dataPoint = DataPoint(t, v, label)
                dataPointsByTime[t] = dataPoint
            self.metricsByLabel[label] = dataPointsByTime

class TimeSeries:
    def __init__(self):
        self.metricsByLabel = {}

    def addDataPointFromFileParser(self, fp):
        for label in fp.metricsByLabel:
            self.metricsByLabel[label] = fp.metricsByLabel[label]
    
    def getDataByTime(self, timeS):
        res = []
        for label, metric in self.metricsByLabel:
            if time in metric:
                res.append(metric[timeS])
        return res


class TimeNode:
    def __init__(self):
        self.prev = None
        self.next = None
        self.time = None
        self.dataByLabel = {}
    
    def addPrev(self, n):
        self.prev = n

    def addNext(self, n):
        self.next = n

    def addTime(self, t):
        self.time = t

    def addData(self, d):
        self.dataByLabel[d.label] = d


class TimeLinkedList:
    def __init__(self, timeSeries):
        self.timeNodeByTime = {}
        self.root = None
        for label in timeSeries.metricsByLabel:
            m = timeSeries.metricsByLabel[label]
            for t in m:
                dataPoint = m[t]
                if t in self.timeNodeByTime:
                    self.timeNodeByTime[t].addData(dataPoint)
                else:
                    # create a new timeNode
                    timeNode = TimeNode()
                    timeNode.addTime(t)
                    timeNode.addData(dataPoint)

                    if self.root is None:
                        self.root = timeNode
                        self.timeNodeByTime[t] = timeNode
                    else:
                        self.add(timeNode)

    def add(self, node):
        finding = True
        if self.root.time > node.time:
            self.root.prev = node
            node.next = self.root
            self.root = node
            finding = False
        cNode = self.root
        # print(self.root.time, node.time)
        while finding:
            if (cNode.time < node.time and cNode.next is None):
                cNode.next = node
                node.prev = cNode 
                finding = False 
            elif cNode.time < node.time and node.time < cNode.next.time:
                finding = False
                child = cNode.next
                cNode.next = node
                node.prev = cNode 
                child.prev = node
                node.next = child
            else:
                cNode = cNode.next
        self.timeNodeByTime[node.time] = node
                
                

                             



In [105]:
def parseData(filePath, prefix):
    jsonObj = read_file(filePath)
    fileParser = FileParser()
    fileParser.parse(jsonObj, prefix)
    
    return fileParser

In [106]:
db_data = parseData('./db_data.txt', 'db-')
client_data = parseData('./client_data.txt', 'client-')
server_data = parseData('./server_data.txt', 'server-')

timeSeries = TimeSeries()
timeSeries.addDataPointFromFileParser(db_data)
timeSeries.addDataPointFromFileParser(client_data)
timeSeries.addDataPointFromFileParser(server_data)

timeLinkedList = TimeLinkedList(timeSeries)


In [107]:
#  assume i have a rule set to run in the form of the
d1 = toDateTime('2021-06-15T19:51:00+00:00')
d2 = toDateTime('2021-06-15T19:52:00+00:00')

In [108]:
print(type(d2-d1))
deltaD = d2 - d1

<class 'datetime.timedelta'>


In [109]:
def f (a=1, b=2):
    print(a, b)

f(b=3)

1 3


In [110]:
cNode =timeLinkedList.root
while cNode is not None:
    print(cNode.time)
    cNode = cNode.next

2021-06-15 10:20:00
2021-06-15 10:21:00
2021-06-15 10:22:00
2021-06-15 10:23:00
2021-06-15 10:24:00
2021-06-15 10:25:00
2021-06-15 10:26:00
2021-06-15 10:27:00
2021-06-15 10:28:00
2021-06-15 10:29:00
2021-06-15 10:30:00
2021-06-15 10:31:00
2021-06-15 10:32:00
2021-06-15 10:33:00
2021-06-15 10:34:00
2021-06-15 10:35:00
2021-06-15 10:36:00
2021-06-15 10:37:00
2021-06-15 10:38:00
2021-06-15 10:39:00
2021-06-15 10:40:00
2021-06-15 10:41:00
2021-06-15 10:42:00
2021-06-15 10:43:00
2021-06-15 10:44:00
2021-06-15 10:45:00
2021-06-15 10:46:00
2021-06-15 10:47:00
2021-06-15 10:48:00
2021-06-15 10:49:00
2021-06-15 10:50:00
2021-06-15 10:51:00
2021-06-15 10:52:00
2021-06-15 10:53:00
2021-06-15 10:54:00
2021-06-15 10:55:00
2021-06-15 10:56:00
2021-06-15 10:57:00
2021-06-15 10:58:00
2021-06-15 10:59:00
2021-06-15 11:00:00
2021-06-15 11:01:00
2021-06-15 11:02:00
2021-06-15 11:03:00
2021-06-15 11:04:00
2021-06-15 11:05:00
2021-06-15 11:06:00
2021-06-15 11:07:00
2021-06-15 11:08:00
2021-06-15 11:09:00
